In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os

os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]= "true"

import numpy as np
from tensorflow.keras.optimizers import Adam , RMSprop
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import ModelCheckpoint , EarlyStopping , CSVLogger
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import tensorflow as tf
from skimage import morphology

from tqdm.notebook import tqdm

import requests
import json

In [ ]:
#値を-1から1に正規化する関数
def normalize_x(image):
    return image / 127.5 - 1


#値を0から1正規化する関数
def normalize_y(image):
    return image / 255


#値を0から255に戻す関数
def denormalize_y(image):
    return image * 255

In [ ]:
# インプット画像を読み込む関数
def load_X_gray(folder_path):
    
    image_files = []

    #image_files = os.listdir(folder_path)
       
    for file in os.listdir(folder_path):
        base, ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(file)
        else :
            pass
        
    image_files.sort()
    
    img = cv2.imread(folder_path + os.sep + image_files[0], cv2.IMREAD_GRAYSCALE)
    
    #image_files = image_files[1:]
    images = np.zeros((len(image_files), img.shape[0], img.shape[1], 1), np.float32)
    for i, image_file in tqdm(enumerate(image_files)):
        image = cv2.imread(folder_path + os.sep + image_file, cv2.IMREAD_GRAYSCALE)
        #image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
        image = image[:, :, np.newaxis]
        images[i] = normalize_x(image)
    
    print(images.shape)
    
    return images, image_files


def load_Y_gray(folder_path, thresh = None , normalize = False, g_size = None):
    image_files = []
    #image_files = os.listdir(folder_path)
    
    for file in os.listdir(folder_path):
        base, ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(file)
        else:
            pass
        
    image_files.sort()
    
    img = cv2.imread(folder_path + os.sep + image_files[0], cv2.IMREAD_GRAYSCALE)
    
    images = np.zeros(
        (len(image_files), img.shape[0], img.shape[1], 1) ,np.float32
    )
    
    for i , image_file in tqdm(enumerate(image_files)):
        image = cv2.imread(
            folder_path + os.sep + image_file ,
            cv2.IMREAD_GRAYSCALE
        )
        #print(image.shape)
        
        # ぼかし処理
        if g_size:
            image = cv2.GaussianBlur(
                image, (g_size, g_size), 0
            )        
        
        if thresh:
            ret , image = cv2.threshold(image , thresh , 255 , cv2.THRESH_BINARY)
        image = image[ : , : , np.newaxis]
        if normalize:
            images[i] = normalize_y(image)
        else:
            images[i] = image
            
    print(images.shape)
    
    return images , image_files

In [ ]:
def dice_coeff(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score


def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss


def bce_dice_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss


def weighted_dice_coeff(y_true, y_pred, weight):
    smooth = 1.
    w, m1, m2 = weight * weight, y_true, y_pred
    intersection = (m1 * m2)
    score = (2. * K.sum(w * intersection) + smooth) / (K.sum(w * m1) + K.sum(w * m2) + smooth)
    return score


def weighted_dice_loss(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    # if we want to get same size of output, kernel size must be odd number
    if K.int_shape(y_pred)[1] == 128:
        kernel_size = 11
    elif K.int_shape(y_pred)[1] == 256:
        kernel_size = 21
    elif K.int_shape(y_pred)[1] == 512:
        kernel_size = 21
    elif K.int_shape(y_pred)[1] == 1024:
        kernel_size = 41
    else:
        raise ValueError('Unexpected image size')
    averaged_mask = K.pool2d(
        y_true, pool_size=(kernel_size, kernel_size), strides=(1, 1), padding='same', pool_mode='avg')
    border = K.cast(K.greater(averaged_mask, 0.005), 'float32') * K.cast(K.less(averaged_mask, 0.995), 'float32')
    weight = K.ones_like(averaged_mask)
    w0 = K.sum(weight)
    weight += border * 2
    w1 = K.sum(weight)
    weight *= (w0 / w1)
    loss = 1 - weighted_dice_coeff(y_true, y_pred, weight)
    return loss


def weighted_bce_loss(y_true, y_pred, weight):
    # avoiding overflow
    epsilon = 1e-7
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
    logit_y_pred = K.log(y_pred / (1. - y_pred))

    # https://www.tensorflow.org/api_docs/python/tf/nn/weighted_cross_entropy_with_logits
    loss = (1. - y_true) * logit_y_pred + (1. + (weight - 1.) * y_true) * \
                                          (K.log(1. + K.exp(-K.abs(logit_y_pred))) + K.maximum(-logit_y_pred, 0.))
    return K.sum(loss) / K.sum(weight)


def weighted_bce_dice_loss(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    # if we want to get same size of output, kernel size must be odd number
    if K.int_shape(y_pred)[1] == 128:
        kernel_size = 11
    elif K.int_shape(y_pred)[1] == 256:
        kernel_size = 21
    elif K.int_shape(y_pred)[1] == 512:
        kernel_size = 21
    elif K.int_shape(y_pred)[1] == 1024:
        kernel_size = 41
    else:
        raise ValueError('Unexpected image size')
    averaged_mask = K.pool2d(
        y_true, pool_size=(kernel_size, kernel_size), strides=(1, 1), padding='same', pool_mode='avg')
    border = K.cast(K.greater(averaged_mask, 0.005), 'float32') * K.cast(K.less(averaged_mask, 0.995), 'float32')
    weight = K.ones_like(averaged_mask)
    w0 = K.sum(weight)
    weight += border * 2
    w1 = K.sum(weight)
    weight *= (w0 / w1)
    loss = weighted_bce_loss(y_true, y_pred, weight) + (1 - weighted_dice_coeff(y_true, y_pred, weight))
    return loss

In [ ]:
def get_unet_resnet_512(input_shape=(512, 512, 1), num_classes=3):
    
    inputs = Input(shape=input_shape)
    
    res0 = Conv2D(16, (1, 1), padding = 'same',  use_bias = False)(inputs)
    down0 = Conv2D(16, (3, 3), padding='same')(inputs)
    down0 = BatchNormalization()(down0)
    down0 = Activation('relu')(down0)
    down0 = Conv2D(16, (3, 3), padding='same')(down0)
    down0 = BatchNormalization()(down0)
    down0 = Add()([res0, down0])
    down0 = Activation('relu')(down0)
    down0_pool = MaxPooling2D((2, 2), strides=(2, 2))(down0)
    
    res1 = Conv2D(32, (1, 1), padding = 'same',  use_bias = False)(down0_pool)
    down1 = Conv2D(32, (3, 3), padding='same')(down0_pool)
    down1 = BatchNormalization()(down1)
    down1 = Activation('relu')(down1)
    down1 = Conv2D(32, (3, 3), padding='same')(down1)
    down1 = BatchNormalization()(down1)
    down1 = Add()([res1, down1])
    down1 = Activation('relu')(down1)
    down1_pool = MaxPooling2D((2, 2), strides=(2, 2))(down1)
    
    res2 = Conv2D(64, (1, 1), padding = 'same',  use_bias = False)(down1_pool)
    down2 = Conv2D(64, (3, 3), padding='same')(down1_pool)
    down2 = BatchNormalization()(down2)
    down2 = Activation('relu')(down2)
    down2 = Conv2D(64, (3, 3), padding='same')(down2)
    down2 = BatchNormalization()(down2)
    down2 = Add()([res2, down2])
    down2 = Activation('relu')(down2)
    down2_pool = MaxPooling2D((2, 2), strides=(2, 2))(down2)
    
    res3 = Conv2D(128, (1, 1), padding = 'same',  use_bias = False)(down2_pool)
    down3 = Conv2D(128, (3, 3), padding='same')(down2_pool)
    down3 = BatchNormalization()(down3)
    down3 = Activation('relu')(down3)
    down3 = Conv2D(128, (3, 3), padding='same')(down3)
    down3 = BatchNormalization()(down3)
    down3 = Add()([res3, down3])
    down3 = Activation('relu')(down3)
    down3_pool = MaxPooling2D((2, 2), strides=(2, 2))(down3)
    
    res4 = Conv2D(256, (1, 1), padding = 'same',  use_bias = False)(down3_pool)
    down4 = Conv2D(256, (3, 3), padding='same')(down3_pool)
    down4 = BatchNormalization()(down4)
    down4 = Activation('relu')(down4)
    down4 = Conv2D(256, (3, 3), padding='same')(down4)
    down4 = BatchNormalization()(down4)
    down4 = Add()([res4, down4])
    down4 = Activation('relu')(down4)
    down4_pool = MaxPooling2D((2, 2), strides=(2, 2))(down4)
    
    res5 = Conv2D(512, (1, 1), padding = 'same',  use_bias = False)(down4_pool)
    down5 = Conv2D(512, (3, 3), padding='same')(down4_pool)
    down5 = BatchNormalization()(down5)
    down5 = Activation('relu')(down5)
    down5 = Conv2D(512, (3, 3), padding='same')(down5)
    down5 = BatchNormalization()(down5)
    down5 = Add()([res5, down5])
    down5 = Activation('relu')(down5)
    down5_pool = MaxPooling2D((2, 2), strides=(2, 2))(down5)
    
    res6 = Conv2D(1024, (1, 1), padding = 'same',  use_bias = False)(down5_pool)
    center = Conv2D(1024, (3, 3), padding='same')(down5_pool)
    center = BatchNormalization()(center)
    center = Activation('relu')(center)
    center = Conv2D(1024, (3, 3), padding='same')(center)
    center = BatchNormalization()(center)
    center = Add()([res6, center])
    center = Activation('relu')(center)
    
    up5 = UpSampling2D((2, 2))(center)
    up5 = Conv2D(512, (2, 2), padding = 'same')(up5)
    up5 = Activation('relu')(up5)
    up5 = concatenate([up5, down5], axis = 3)
    res_up5 = Conv2D(512, (1, 1), padding = 'same', use_bias = False)(up5)
    up5 = Conv2D(512, (3, 3), padding = 'same')(up5)
    up5 = BatchNormalization()(up5)
    up5 = Activation('relu')(up5)
    up5 = Conv2D(512, (3, 3), padding = 'same')(up5)
    up5 = BatchNormalization()(up5)
    up5 = Add()([res_up5, up5])
    up5 = Activation('relu')(up5)
    
    up4 = UpSampling2D((2, 2))(up5)
    up4 = Conv2D(256, (2, 2), padding = 'same')(up4)
    up4 = Activation('relu')(up4)
    up4 = concatenate([up4, down4], axis = 3)
    res_up4 = Conv2D(256, (1, 1), padding = 'same', use_bias = False)(up4)
    up4 = Conv2D(256, (3, 3), padding = 'same')(up4)
    up4 = BatchNormalization()(up4)
    up4 = Activation('relu')(up4)
    up4 = Conv2D(256, (3, 3), padding = 'same')(up4)
    up4 = BatchNormalization()(up4)
    up4 = Add()([res_up4, up4])
    up4 = Activation('relu')(up4)
    
    up3 = UpSampling2D((2, 2))(up4)
    up3 = Conv2D(128, (2, 2), padding = 'same')(up3)
    up3 = Activation('relu')(up3)
    up3 = concatenate([up3, down3], axis = 3)
    res_up3 = Conv2D(128, (1, 1), padding = 'same', use_bias = False)(up3)
    up3 = Conv2D(128, (3, 3), padding = 'same')(up3)
    up3 = BatchNormalization()(up3)
    up3 = Activation('relu')(up3)
    up3 = Conv2D(128, (3, 3), padding = 'same')(up3)
    up3 = BatchNormalization()(up3)
    up3 = Add()([res_up3, up3])
    up3 = Activation('relu')(up3)
    
    up2 = UpSampling2D((2, 2))(up3)
    up2 = Conv2D(64, (2, 2), padding = 'same')(up2)
    up2 = Activation('relu')(up2)
    up2 = concatenate([up2, down2], axis = 3)
    res_up2 = Conv2D(64, (1, 1), padding = 'same', use_bias = False)(up2)
    up2 = Conv2D(64, (3, 3), padding = 'same')(up2)
    up2 = BatchNormalization()(up2)
    up2 = Activation('relu')(up2)
    up2 = Conv2D(64, (3, 3), padding = 'same')(up2)
    up2 = BatchNormalization()(up2)
    up2 = Add()([res_up2, up2])
    up2 = Activation('relu')(up2)
    
    up1 = UpSampling2D((2, 2))(up2)
    up1 = Conv2D(32, (2, 2), padding = 'same')(up1)
    up1 = Activation('relu')(up1)
    up1 = concatenate([up1, down1], axis = 3)
    res_up1 = Conv2D(32, (1, 1), padding = 'same', use_bias = False)(up1)
    up1 = Conv2D(32, (3, 3), padding = 'same')(up1)
    up1 = BatchNormalization()(up1)
    up1 = Activation('relu')(up1)
    up1 = Conv2D(32, (3, 3), padding = 'same')(up1)
    up1 = BatchNormalization()(up1)
    up1 = Add()([res_up1, up1])
    up1 = Activation('relu')(up1)
    
    up0 = UpSampling2D((2, 2))(up1)
    up0 = Conv2D(16, (2, 2), padding = 'same')(up0)
    up0 = Activation('relu')(up0)
    up0 = concatenate([up0, down0], axis = 3)
    res_up0 = Conv2D(16, (1, 1), padding = 'same', use_bias = False)(up0)
    up0 = Conv2D(16, (3, 3), padding = 'same')(up0)
    up0 = BatchNormalization()(up0)
    up0 = Activation('relu')(up0)
    up0 = Conv2D(16, (3, 3), padding = 'same')(up0)
    up0 = BatchNormalization()(up0)
    up0 = Add()([res_up0, up0])
    up0 = Activation('relu')(up0)
    
    classify = Conv2D(num_classes, (1, 1), activation='sigmoid')(up0)
    #classify = Conv2D(num_classes, (1, 1), padding = 'same', activation='softmax')(up0)
    model = Model(inputs=inputs, outputs=classify)
    
    model.compile(optimizer=RMSprop(lr=0.0001), loss=bce_dice_loss, metrics=[dice_coeff])
    
    return model

In [ ]:
def divide_rgb_imgs(images):
    
    '''
        512 x 512に分割する
        端の50pxをのりしろとする
        空欄箇所は-1（黒）にする
        height, widthが412未満の場合は変数を-1に変更
    '''
    
    H = -(-images.shape[1]//412)
    W = -(-images.shape[2]//412)
    
    diveded_imgs = np.zeros(( images.shape[0]*H*W, 512, 512, 1), np.float32)
    print(H,W)
    
    for z in range(images.shape[0]):
        image = images[z]
        for h in range(H):
            for w in range(W):
                cropped_img = np.zeros((512, 512, 1), np.float32)
                #cropped_img -= 1
                
                if images.shape[1] < 412:
                    h = -1
                if images.shape[2] < 412:
                    w = -1
                    
                if h == -1:
                    if w == -1:
                        cropped_img[50:images.shape[1]+50, 50:images.shape[2]+50, 0] = image[0:images.shape[1], 0:images.shape[2], 0]
                    elif w == 0:
                        cropped_img[50:images.shape[1]+50, 50:512, 0] = image[0:images.shape[1], 0:462, 0]
                    elif w == W-1:
                        cropped_img[50:images.shape[1]+50, 0:images.shape[2]-412*W-50, 0] = image[0:images.shape[1], w*412-50:images.shape[2], 0]
                    else:
                        cropped_img[50:images.shape[1]+50, :, 0] = image[0:images.shape[1], w*412-50:(w+1)*412+50, 0]
                elif h == 0:
                    if w == -1:
                        cropped_img[50:512, 50:images.shape[2]+50, 0] = image[0:462, 0:images.shape[2], 0]
                    elif w == 0:
                        cropped_img[50:512, 50:512, 0] = image[0:462, 0:462, 0]
                    elif w == W-1:
                        cropped_img[50:512, 0:images.shape[2]-412*W-50, 0] = image[0:462, w*412-50:images.shape[2], 0]
                    else:
                        #cropped_img[50:512, :, 0] = image[0:462, w*412-50:(w+1)*412+50, 0]
                        try:
                            cropped_img[50:512, :, 0] = image[0:462, w*412-50:(w+1)*412+50, 0]
                        except:
                            cropped_img[50:512, 0:images.shape[2]-412*(W-1)-50, 0] = image[0:462, w*412-50:(w+1)*412+50, 0]
                elif h == H-1:
                    if w == -1:
                        cropped_img[0:images.shape[1]-412*H-50, 50:images.shape[2]+50, 0] = image[h*412-50:images.shape[1], 0:images.shape[2], 0]
                    elif w == 0:
                        cropped_img[0:images.shape[1]-412*H-50, 50:512, 0] = image[h*412-50:images.shape[1], 0:462, 0]
                    elif w == W-1:
                        cropped_img[0:images.shape[1]-412*H-50, 0:images.shape[2]-412*W-50, 0] = image[h*412-50:images.shape[1], w*412-50:images.shape[2], 0]
                    else:
                        try:
                            cropped_img[0:images.shape[1]-412*H-50, :, 0] = image[h*412-50:images.shape[1], w*412-50:(w+1)*412+50, 0]
                        except:
                            cropped_img[0:images.shape[1]-412*H-50, 0:images.shape[2]-412*(W-1)-50, 0] = image[h*412-50:images.shape[1], w*412-50:(w+1)*412+50, 0]
                else:
                    if w == -1:
                        cropped_img[:, 50:images.shape[2]+50, 0] = image[h*412-50:(h+1)*412+50, 0:images.shape[2], 0]
                    elif w == 0:
                        #cropped_img[:, 50:512, 0] = image[h*412-50:(h+1)*412+50, 0:462, 0]
                        try:
                            cropped_img[:, 50:512, 0] = image[h*412-50:(h+1)*412+50, 0:462, 0]
                        except:
                            cropped_img[0:images.shape[1]-412*H-50+412, 50:512, 0] = image[h*412-50:(h+1)*412+50, 0:462, 0]
                    elif w == W-1:
                        #cropped_img[:, 0:images.shape[2]-412*W-50, 0] = image[h*412-50:(h+1)*412+50, w*412-50:images.shape[2], 0]
                        try:
                            cropped_img[:, 0:images.shape[2]-412*W-50, 0] = image[h*412-50:(h+1)*412+50, w*412-50:images.shape[2], 0]
                        except:
                            cropped_img[0:images.shape[1]-412*H-50+412, 0:images.shape[2]-412*W-50, 0] = image[h*412-50:(h+1)*412+50, w*412-50:images.shape[2], 0]
                    else:
                        #cropped_img[:, :, 0] = image[h*412-50:(h+1)*412+50, w*412-50:(w+1)*412+50, 0]
                        try:
                            cropped_img[:, :, 0] = image[h*412-50:(h+1)*412+50, w*412-50:(w+1)*412+50, 0]         
                        except:
                            try:
                                 cropped_img[:, 0:images.shape[2]-412*(W-1)-50, 0] = image[h*412-50:(h+1)*412+50, w*412-50:(w+1)*412+50, 0]
                            except:
                                 cropped_img[0:images.shape[1]-412*(H-1)-50, :, 0] = image[h*412-50:(h+1)*412+50, w*412-50:(w+1)*412+50, 0]
                h = max(0, h)
                w = max(0, w)
                diveded_imgs[z*H*W+ w*H+h] = cropped_img
                #print(z*H*W+ w*H+h)
                
    return diveded_imgs
    



def divide_gray_imgs(images):
    
    '''
        512 x 512に分割する
        端の50pxをのりしろとする
        空欄箇所は-1（黒）にする
        height, widthが412未満の場合は変数を-1に変更
    '''
    
    H = -(-images.shape[1]//412)
    W = -(-images.shape[2]//412)
    
    diveded_imgs = np.zeros(( images.shape[0]*H*W, 512, 512, 1), np.float32)
    print(H,W)
    
    for z in range(images.shape[0]):
        image = images[z]
        for h in range(H):
            for w in range(W):
                cropped_img = np.zeros((512, 512, 1), np.float32)
                cropped_img -= 1
                
                if images.shape[1] < 412:
                    h = -1
                if images.shape[2] < 412:
                    w = -1
                    
                if h == -1:
                    if w == -1:
                        cropped_img[50:images.shape[1]+50, 50:images.shape[2]+50, 0] = image[0:images.shape[1], 0:images.shape[2], 0]
                    elif w == 0:
                        cropped_img[50:images.shape[1]+50, 50:512, 0] = image[0:images.shape[1], 0:462, 0]
                    elif w == W-1:
                        cropped_img[50:images.shape[1]+50, 0:images.shape[2]-412*W-50, 0] = image[0:images.shape[1], w*412-50:images.shape[2], 0]
                    else:
                        cropped_img[50:images.shape[1]+50, :, 0] = image[0:images.shape[1], w*412-50:(w+1)*412+50, 0]
                elif h == 0:
                    if w == -1:
                        cropped_img[50:512, 50:images.shape[2]+50, 0] = image[0:462, 0:images.shape[2], 0]
                    elif w == 0:
                        cropped_img[50:512, 50:512, 0] = image[0:462, 0:462, 0]
                    elif w == W-1:
                        cropped_img[50:512, 0:images.shape[2]-412*W-50, 0] = image[0:462, w*412-50:images.shape[2], 0]
                    else:
                        #cropped_img[50:512, :, 0] = image[0:462, w*412-50:(w+1)*412+50, 0]
                        try:
                            cropped_img[50:512, :, 0] = image[0:462, w*412-50:(w+1)*412+50, 0]
                        except:
                            cropped_img[50:512, 0:images.shape[2]-412*(W-1)-50, 0] = image[0:462, w*412-50:(w+1)*412+50, 0]
                elif h == H-1:
                    if w == -1:
                        cropped_img[0:images.shape[1]-412*H-50, 50:images.shape[2]+50, 0] = image[h*412-50:images.shape[1], 0:images.shape[2], 0]
                    elif w == 0:
                        cropped_img[0:images.shape[1]-412*H-50, 50:512, 0] = image[h*412-50:images.shape[1], 0:462, 0]
                    elif w == W-1:
                        cropped_img[0:images.shape[1]-412*H-50, 0:images.shape[2]-412*W-50, 0] = image[h*412-50:images.shape[1], w*412-50:images.shape[2], 0]
                    else:
                        try:
                            cropped_img[0:images.shape[1]-412*H-50, :, 0] = image[h*412-50:images.shape[1], w*412-50:(w+1)*412+50, 0]
                        except:
                            cropped_img[0:images.shape[1]-412*H-50, 0:images.shape[2]-412*(W-1)-50, 0] = image[h*412-50:images.shape[1], w*412-50:(w+1)*412+50, 0]
                else:
                    if w == -1:
                        cropped_img[:, 50:images.shape[2]+50, 0] = image[h*412-50:(h+1)*412+50, 0:images.shape[2], 0]
                    elif w == 0:
                        #cropped_img[:, 50:512, 0] = image[h*412-50:(h+1)*412+50, 0:462, 0]
                        try:
                            cropped_img[:, 50:512, 0] = image[h*412-50:(h+1)*412+50, 0:462, 0]
                        except:
                            cropped_img[0:images.shape[1]-412*H-50+412, 50:512, 0] = image[h*412-50:(h+1)*412+50, 0:462, 0]
                    elif w == W-1:
                        #cropped_img[:, 0:images.shape[2]-412*W-50, 0] = image[h*412-50:(h+1)*412+50, w*412-50:images.shape[2], 0]
                        try:
                            cropped_img[:, 0:images.shape[2]-412*W-50, 0] = image[h*412-50:(h+1)*412+50, w*412-50:images.shape[2], 0]
                        except:
                            cropped_img[0:images.shape[1]-412*H-50+412, 0:images.shape[2]-412*W-50, 0] = image[h*412-50:(h+1)*412+50, w*412-50:images.shape[2], 0]
                    else:
                        #cropped_img[:, :, 0] = image[h*412-50:(h+1)*412+50, w*412-50:(w+1)*412+50, 0]
                        try:
                            cropped_img[:, :, 0] = image[h*412-50:(h+1)*412+50, w*412-50:(w+1)*412+50, 0]         
                        except:
                            try:
                                 cropped_img[:, 0:images.shape[2]-412*(W-1)-50, 0] = image[h*412-50:(h+1)*412+50, w*412-50:(w+1)*412+50, 0]
                            except:
                                 cropped_img[0:images.shape[1]-412*(H-1)-50, :, 0] = image[h*412-50:(h+1)*412+50, w*412-50:(w+1)*412+50, 0]
                h = max(0, h)
                w = max(0, w)
                diveded_imgs[z*H*W+ w*H+h] = cropped_img
                #print(z*H*W+ w*H+h)
                
    return diveded_imgs


def divide_imgs(images):
    if images.shape[3] == 1:
        return divide_gray_imgs(images)
    else:
        return np.concatenate( list( divide_rgb_imgs(images[:,:,:,c][..., np.newaxis]) for c in range(images.shape[3]) ), axis = 3 )
            
    
    
def merge_imgs(imgs, original_image_shape):
    
    '''
        元のサイズに合体させる
        original_image_shapeはoriginal_image.shapeで与えられる、(z, h, w, channel)のタプル型
    '''
    
    merged_imgs = np.zeros((original_image_shape[0], original_image_shape[1], original_image_shape[2], 1), np.float32)
    H = -(-original_image_shape[1]//412)
    W = -(-original_image_shape[2]//412)    
    
    for z in range(original_image_shape[0]):
        for h in range(H):
            for w in range(W):

                if original_image_shape[1] < 412:
                    h = -1
                if original_image_shape[2] < 412:
                    w = -1
                    
                #print(z*H*W+ max(w, 0)*H+max(h, 0))    
                if h == -1:
                    if w == -1:
                        merged_imgs[z, 0:original_image_shape[1], 0:original_image_shape[2], 0] = imgs[z*H*W+ 0*H+0][50:original_image_shape[1]+50, 50:original_image_shape[2]+50, 0]
                    elif w == 0:
                        merged_imgs[z, 0:original_image_shape[1], 0:412, 0] = imgs[z*H*W+ w*H+0][50:original_image_shape[1]+50, 50:462, 0]
                    elif w == W-1:
                        merged_imgs[z, 0:original_image_shape[1], w*412:original_image_shape[2], 0] = imgs[z*H*W+ w*H+0][50:original_image_shape[1]+50, 50:original_image_shape[2]-412*W-50, 0]
                    else:
                        merged_imgs[z, 0:original_image_shape[1], w*412:(w+1)*412, 0] = imgs[z*H*W+ w*H+0][50:original_image_shape[1]+50, 50:462, 0]
                elif h == 0:
                    if w == -1:
                        merged_imgs[z, 0:412, 0:original_image_shape[2], 0] = imgs[z*H*W+ 0*H+h][50:462, 50:original_image_shape[2]+50, 0]
                    elif w == 0:
                        merged_imgs[z, 0:412, 0:412, 0] = imgs[z*H*W+ w*H+h][50:462, 50:462, 0]
                    elif w == W-1:
                        merged_imgs[z, 0:412, w*412:original_image_shape[2], 0] = imgs[z*H*W+ w*H+h][50:462, 50:original_image_shape[2]-412*W-50, 0]
                    else:
                        merged_imgs[z, 0:412, w*412:(w+1)*412, 0] = imgs[z*H*W+ w*H+h][50:462, 50:462, 0]
                elif h == H-1:
                    if w == -1:
                         merged_imgs[z, h*412:original_image_shape[1], 0:original_image_shape[2], 0] = imgs[z*H*W+ 0*H+h][50:original_image_shape[1]-412*H-50, 50:original_image_shape[2]+50, 0]
                    elif w == 0:
                        merged_imgs[z, h*412:original_image_shape[1], 0:412, 0] = imgs[z*H*W+ w*H+h][50:original_image_shape[1]-412*H-50, 50:462, 0]
                    elif w == W-1:
                        merged_imgs[z, h*412:original_image_shape[1], w*412:original_image_shape[2], 0] = imgs[z*H*W+ w*H+h][50:original_image_shape[1]-412*H-50, 50:original_image_shape[2]-412*W-50, 0]
                    else:
                        merged_imgs[z, h*412:original_image_shape[1], w*412:(w+1)*412, 0] = imgs[z*H*W+ w*H+h][50:original_image_shape[1]-412*H-50, 50:462, 0]
                else:
                    if w == -1:
                         merged_imgs[z, h*412:(h+1)*412, 0:original_image_shape[2], 0] = imgs[z*H*W+ 0*H+h][50:462, 50:original_image_shape[2]+50, 0]
                    elif w == 0:
                        merged_imgs[z, h*412:(h+1)*412, 0:412, 0] = imgs[z*H*W+ w*H+h][50:462, 50:462, 0]
                    elif w == W-1:
                        merged_imgs[z, h*412:(h+1)*412, w*412:original_image_shape[2], 0] = imgs[z*H*W+ w*H+h][50:462, 50:original_image_shape[2]-412*W-50, 0]
                    else:
                        merged_imgs[z, h*412:(h+1)*412, w*412:(w+1)*412, 0] = imgs[z*H*W+ w*H+h][50:462, 50:462, 0]  
                        
    return merged_imgs


def load_Y_gray_with_gaussian(folder_path, thresh = None , normalize = True, g_size = None):
    image_files = []
    #image_files = os.listdir(folder_path)
    
    for file in os.listdir(folder_path):
        base, ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(file)
        else:
            pass
        
    image_files.sort()
    
    img = cv2.imread(folder_path + os.sep + image_files[0], cv2.IMREAD_GRAYSCALE)
    
    images = np.zeros(
        (len(image_files), img.shape[0], img.shape[1], 1) ,np.float32
    )
    
    for i , image_file in enumerate(image_files):
        image = cv2.imread(
            folder_path + os.sep + image_file ,
            cv2.IMREAD_GRAYSCALE
        )
        
        # ぼかし処理
        if g_size:
            image = cv2.GaussianBlur(
                image, (g_size, g_size), 0
            )
        
        if thresh:
            ret , image = cv2.threshold(image , thresh , 255 , cv2.THRESH_BINARY)
        image = image[ : , : , np.newaxis]
        if normalize:
            images[i] = normalize_y(image)
        else:
            images[i] = image
            
    print(images.shape)
    
    return images , image_files

In [ ]:
def crop_80_percents_data(imgs):
    
    """
        Args : 
            imgs (numpy.ndarray) : Z, Y, X, 1
        
        Returns :
            imgs (numpy.ndarray) : Z, Y, X, 1
            
        Memo :
            z, y, x　の全ての上下左右縦横10%を捨てる
    """
    z_range = [int(imgs.shape[0] * 0.1), int(imgs.shape[0] * 0.9)]
    y_range = [int(imgs.shape[1] * 0.1), int(imgs.shape[1] * 0.9)]
    x_range = [int(imgs.shape[2] * 0.1), int(imgs.shape[2] * 0.9)]
    
    return imgs[z_range[0]:z_range[1], y_range[0]:y_range[1], x_range[0]:x_range[1], :]


def make_cristae_train_datasets(ori_images, label_images, num_list):
    
    """
        Args : 
            ori_images (numpy.ndarray) : Z, Y, X, 1
            label_images (numpy.ndarray) : Z, Y, X, 1
            num_list (list) : num
            
        Returns : 
            train_ori_images (numpy.ndarray) : Z, Y, X, 1
            train_label_images (numpy.ndarray) : Z, Y, X, 2
            
        Memo:
            num_listに入っている画像を抽出
            label_images は "1:lamellar, 2:tubular"
        
    """
    
    train_ori_images = list()
    train_label_images = list()
    for num in num_list:
        ori_image = ori_images[num]
        label_image = label_images[num]
        
        # 切り出す範囲を指定
        x_range = [0, ori_image.shape[0]]
        y_range = [0, ori_image.shape[1]]
        
        train_ori_images.append(ori_image[x_range[0]:x_range[1], y_range[0]:y_range[1], :])
        train_label_images.append(label_image[x_range[0]:x_range[1], y_range[0]:y_range[1], :])
        
    train_label_images_001 = np.where(
        np.array(train_label_images) == 1,
        1,
        0
    )
    train_label_images_002 = np.where(
        np.array(train_label_images) == 2,
        1,
        0
    )
        
    cropped_ori_images = divide_gray_imgs(np.array(train_ori_images))
    cropped_label_images_001 = divide_gray_imgs(np.array(train_label_images_001))
    cropped_label_images_002 = divide_gray_imgs(np.array(train_label_images_002))
    
    cropped_ori_images = np.where(
        cropped_ori_images == -1,
        1,
        cropped_ori_images
    )
    
    cropped_label_images_001 = np.where(
        cropped_label_images_001 == -1,
        0,
        cropped_label_images_001
    )
    
    cropped_label_images_002 = np.where(
        cropped_label_images_002 == -1,
        0,
        cropped_label_images_002
    )
    cropped_label_images = np.concatenate((cropped_label_images_001, cropped_label_images_002, np.zeros(cropped_label_images_001.shape)), axis = 3)
        
    return cropped_ori_images, cropped_label_images

In [ ]:
def make_white_mask_ori_img(ori_img, mask_img):
    
    masked_img = np.zeros(
        (ori_img.shape[0], ori_img.shape[1], ori_img.shape[2], 1),
        np.float32
    )
    print(masked_img.shape)
    print(ori_img.shape)
    print(mask_img.shape)
    
    for i in range(ori_img.shape[0]):
        masked_img[:,:,:,0][i] = np.where(
            mask_img[:,:,:,0][i] == 1,
            ori_img[:,:,:,-1][i],
            1
        )
        
    return masked_img

In [ ]:
def make_margin(imgs):
    
    """
        412の倍数のH,Wにする
    """
    
    original_image_shape = imgs.shape
    H = -(-original_image_shape[1]//412)
    W = -(-original_image_shape[2]//412) 
    made_imgs = np.zeros(
        (original_image_shape[0], H*412, W*412, 1), 
        np.float32
    )
    
    start_h = (H*412 - original_image_shape[1]) // 2
    start_w = (W*412 - original_image_shape[2]) // 2
    
    made_imgs[:, start_h:start_h+original_image_shape[1], start_w:start_w+original_image_shape[2], :] = imgs
    return made_imgs


def delete_margin(imgs, original_image_shape):
    
    """
        marginを削除
    """
    
    H = -(-original_image_shape[1]//412)
    W = -(-original_image_shape[2]//412)
    
    start_h = (H*412 - original_image_shape[1]) // 2
    start_w = (W*412 - original_image_shape[2]) // 2
    
    return imgs[:, start_h:start_h+original_image_shape[1], start_w:start_w+original_image_shape[2], :]

In [ ]:
# 学習後のU-Netによる予測を行う関数
def predict(X_test, model_path, filenames, out_dir, input_shape=(512, 512, 1), num_classes=1):

    # testDataフォルダ配下にleft_imagesフォルダを置いている
    
    model = get_unet_resnet_512(input_shape=input_shape, num_classes=num_classes)
    
    model.load_weights(model_path)
    BATCH_SIZE = 1
    Y_pred = model.predict(X_test, BATCH_SIZE)
    
    print(Y_pred.shape)
    os.makedirs(out_dir, exist_ok = True)
    
    if Y_pred.shape[3]!=1:
        num = Y_pred.shape[3]
        for n in range(num):
            os.makedirs(os.path.join(out_dir,str(n+1)), exist_ok=True)
        for i, y in enumerate(Y_pred):
            for n in range(num):
                os.makedirs(os.path.join(out_dir,str(n+1)), exist_ok=True)
            # testDataフォルダ配下にleft_imagesフォルダを置いている
            #img = cv2.imread('512/hira/For_Seika/learning/original/'+ file_names[i], cv2.IMREAD_GRAYSCALE)
            #y = cv2.resize(y, (img.shape[1], img.shape[0]))
                cv2.imwrite(os.path.join(out_dir, str(n+1) , str(i).zfill(6) + '.png'), denormalize_y(y[:,:,n]))
        
    else:
        for i, y in enumerate(Y_pred):
            # testDataフォルダ配下にleft_imagesフォルダを置いている
            #img = cv2.imread('512/hira/For_Seika/learning/original/'+ file_names[i], cv2.IMREAD_GRAYSCALE)
            #y = cv2.resize(y, (img.shape[1], img.shape[0]))
            cv2.imwrite(os.path.join(out_dir , str(i).zfill(6) + '.png'), denormalize_y(y))

# shCtrl

## cropped_001

### pred_001

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_001/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_001/cristae_active_learning/cristae_unet_resblock_001_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_xy_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_yz_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_zx_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_xy_001/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_yz_001/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_zx_001/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx

In [ ]:
imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/merged_cristae_001/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_xy_001/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_yz_001/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_zx_001/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx

In [ ]:
imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/merged_cristae_001/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/merged_cristae_001/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/merged_cristae_001/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/merged_cristae_001/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )

In [ ]:
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/manually_cristae_001/'
os.makedirs(out_dir, exist_ok = True)

### pred_002

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_001/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_001/cristae_active_learning/cristae_unet_resblock_002_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_xy_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_yz_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_zx_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_xy_002/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_yz_002/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_zx_002/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/merged_cristae_002/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_xy_002/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_yz_002/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_zx_002/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/merged_cristae_002/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/merged_cristae_002/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/merged_cristae_002/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/merged_cristae_002/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/manually_cristae_002/'
os.makedirs(out_dir, exist_ok = True)

### pred_003

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_001/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_001/cristae_active_learning/cristae_unet_resblock_003_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_xy_003/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_yz_003/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_zx_003/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_xy_003/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_yz_003/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_zx_003/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/merged_cristae_003/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_xy_003/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_yz_003/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_zx_003/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/merged_cristae_003/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/merged_cristae_003/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/merged_cristae_003/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/merged_cristae_003/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/manually_cristae_003/'
os.makedirs(out_dir, exist_ok = True)

### pred_004

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_001/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_001/cristae_active_learning/cristae_unet_resblock_004_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_xy_004/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_yz_004/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_zx_004/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_xy_004/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_yz_004/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_zx_004/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/merged_cristae_004/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_xy_004/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_yz_004/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_zx_004/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/merged_cristae_004/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/merged_cristae_004/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/merged_cristae_004/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/merged_cristae_004/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/manually_cristae_004/'
os.makedirs(out_dir, exist_ok = True)

### pred_final

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_001/resize_10x10x10')

ori_image_shape = ori_imgs.shape
del ori_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_xy_004/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_yz_004/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_zx_004/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx

imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/merged_cristae_final/lamellar'
os.makedirs(out_dir, exist_ok = True)

cnt = 0
for i in range(imgs_ave.shape[0]):
    
    # threshed
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_xy_004/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_yz_004/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/cristae_pred_zx_004/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx

imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/merged_cristae_final/tubular'
os.makedirs(out_dir, exist_ok = True)

cnt = 0
for i in range(imgs_ave.shape[0]):
    
    # threshed
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    cnt += 1

    


## cropped_002

### pred_001

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_002/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_002/cristae_active_learning/cristae_unet_resblock_001_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_xy_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_yz_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_zx_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_xy_001/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_yz_001/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_zx_001/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/merged_cristae_001/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_xy_001/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_yz_001/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_zx_001/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/merged_cristae_001/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/merged_cristae_001/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/merged_cristae_001/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/merged_cristae_001/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/manually_cristae_001/'
os.makedirs(out_dir, exist_ok = True)

### pred_002

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_002/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_002/cristae_active_learning/cristae_unet_resblock_002_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_xy_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_yz_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_zx_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_xy_002/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_yz_002/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_zx_002/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/merged_cristae_002/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_xy_002/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_yz_002/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_zx_002/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/merged_cristae_002/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/merged_cristae_002/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/merged_cristae_002/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/merged_cristae_002/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/manually_cristae_002/'
os.makedirs(out_dir, exist_ok = True)

### pred_003

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_002/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_002/cristae_active_learning/cristae_unet_resblock_003_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_xy_003/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_yz_003/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_zx_003/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_xy_003/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_yz_003/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_zx_003/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/merged_cristae_003/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_xy_003/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_yz_003/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_zx_003/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/merged_cristae_003/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/merged_cristae_003/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/merged_cristae_003/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/merged_cristae_003/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/manually_cristae_003/'
os.makedirs(out_dir, exist_ok = True)

### pred_004

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_002/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_002/cristae_active_learning/cristae_unet_resblock_004_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_xy_004/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_yz_004/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_zx_004/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_xy_004/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_yz_004/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_zx_004/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/merged_cristae_004/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_xy_004/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_yz_004/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_zx_004/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/merged_cristae_004/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/merged_cristae_004/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/merged_cristae_004/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/merged_cristae_004/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/manually_cristae_004/'
os.makedirs(out_dir, exist_ok = True)

### pred_final

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_002/resize_10x10x10')

ori_image_shape = ori_imgs.shape
del ori_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_xy_004/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_yz_004/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_zx_004/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx

imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/merged_cristae_final/lamellar'
os.makedirs(out_dir, exist_ok = True)

cnt = 0
for i in range(imgs_ave.shape[0]):
    
    # threshed
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_xy_004/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_yz_004/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/cristae_pred_zx_004/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx

imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/merged_cristae_final/tubular'
os.makedirs(out_dir, exist_ok = True)

cnt = 0
for i in range(imgs_ave.shape[0]):
    
    # threshed
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    cnt += 1

    


## cropped_003

### pred_001

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_003/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_003/cristae_active_learning/cristae_unet_resblock_001_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_xy_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_yz_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_zx_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_xy_001/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_yz_001/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_zx_001/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/merged_cristae_001/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_xy_001/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_yz_001/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_zx_001/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/merged_cristae_001/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/merged_cristae_001/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/merged_cristae_001/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/merged_cristae_001/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/manually_cristae_001/'
os.makedirs(out_dir, exist_ok = True)

### pred_002

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_003/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_003/cristae_active_learning/cristae_unet_resblock_002_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_xy_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_yz_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_zx_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_xy_002/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_yz_002/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_zx_002/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/merged_cristae_002/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_xy_002/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_yz_002/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_zx_002/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/merged_cristae_002/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/merged_cristae_002/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/merged_cristae_002/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/merged_cristae_002/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/manually_cristae_002/'
os.makedirs(out_dir, exist_ok = True)

### pred_003

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_003/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_003/cristae_active_learning/cristae_unet_resblock_003_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_xy_003/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_yz_003/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_zx_003/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_xy_003/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_yz_003/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_zx_003/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/merged_cristae_003/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_xy_003/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_yz_003/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_zx_003/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/merged_cristae_003/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/merged_cristae_003/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/merged_cristae_003/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/merged_cristae_003/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/manually_cristae_003/'
os.makedirs(out_dir, exist_ok = True)

### pred_final

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_003/resize_10x10x10')

ori_image_shape = ori_imgs.shape
del ori_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_xy_003/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_yz_003/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_zx_003/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx

imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/merged_cristae_final/lamellar'
os.makedirs(out_dir, exist_ok = True)

cnt = 0
for i in range(imgs_ave.shape[0]):
    
    # threshed
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_xy_003/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_yz_003/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/cristae_pred_zx_003/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx

imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/merged_cristae_final/tubular'
os.makedirs(out_dir, exist_ok = True)

cnt = 0
for i in range(imgs_ave.shape[0]):
    
    # threshed
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    cnt += 1

    


## cropped_004

### pred_001

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_004/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_004/cristae_active_learning/cristae_unet_resblock_001_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_xy_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_yz_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_zx_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_xy_001/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_yz_001/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_zx_001/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/merged_cristae_001/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_xy_001/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_yz_001/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_zx_001/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/merged_cristae_001/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/merged_cristae_001/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/merged_cristae_001/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/merged_cristae_001/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/manually_cristae_001/'
os.makedirs(out_dir, exist_ok = True)

### pred_002

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_004/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_004/cristae_active_learning/cristae_unet_resblock_002_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_xy_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_yz_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_zx_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_xy_002/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_yz_002/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_zx_002/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/merged_cristae_002/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_xy_002/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_yz_002/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_zx_002/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/merged_cristae_002/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/merged_cristae_002/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/merged_cristae_002/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/merged_cristae_002/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/manually_cristae_002/'
os.makedirs(out_dir, exist_ok = True)

### pred_003

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_004/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_004/cristae_active_learning/cristae_unet_resblock_003_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_xy_003/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_yz_003/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_zx_003/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_xy_003/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_yz_003/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_zx_003/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/merged_cristae_003/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_xy_003/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_yz_003/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_zx_003/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/merged_cristae_003/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/merged_cristae_003/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/merged_cristae_003/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/merged_cristae_003/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/manually_cristae_003/'
os.makedirs(out_dir, exist_ok = True)

### pred_final

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_004/resize_10x10x10')

ori_image_shape = ori_imgs.shape
del ori_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_xy_003/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_yz_003/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_zx_003/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx

imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/merged_cristae_final/lamellar'
os.makedirs(out_dir, exist_ok = True)

cnt = 0
for i in range(imgs_ave.shape[0]):
    
    # threshed
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_xy_003/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_yz_003/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/cristae_pred_zx_003/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx

imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/merged_cristae_final/tubular'
os.makedirs(out_dir, exist_ok = True)

cnt = 0
for i in range(imgs_ave.shape[0]):
    
    # threshed
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    cnt += 1

    


## cropped_005

### pred_001

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_005/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_005/cristae_active_learning/cristae_unet_resblock_001_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_xy_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_yz_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_zx_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_xy_001/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_yz_001/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_zx_001/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/merged_cristae_001/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_xy_001/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_yz_001/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_zx_001/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/merged_cristae_001/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/merged_cristae_001/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/merged_cristae_001/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/merged_cristae_001/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/manually_cristae_001/'
os.makedirs(out_dir, exist_ok = True)

### pred_002

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_005/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_005/cristae_active_learning/cristae_unet_resblock_002_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_xy_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_yz_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_zx_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_xy_002/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_yz_002/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_zx_002/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/merged_cristae_002/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_xy_002/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_yz_002/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_zx_002/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/merged_cristae_002/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/merged_cristae_002/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/merged_cristae_002/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/merged_cristae_002/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/manually_cristae_002/'
os.makedirs(out_dir, exist_ok = True)

### pred_003

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_005/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_005/cristae_active_learning/cristae_unet_resblock_003_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_xy_003/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_yz_003/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_zx_003/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_xy_003/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_yz_003/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_zx_003/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/merged_cristae_003/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_xy_003/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_yz_003/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_zx_003/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/merged_cristae_003/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/merged_cristae_003/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/merged_cristae_003/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/merged_cristae_003/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/manually_cristae_003/'
os.makedirs(out_dir, exist_ok = True)

### pred_final

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_005/resize_10x10x10')

ori_image_shape = ori_imgs.shape
del ori_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_xy_003/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_yz_003/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_zx_003/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx

imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/merged_cristae_final/lamellar'
os.makedirs(out_dir, exist_ok = True)

cnt = 0
for i in range(imgs_ave.shape[0]):
    
    # threshed
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_xy_003/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_yz_003/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/cristae_pred_zx_003/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx

imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/merged_cristae_final/tubular'
os.makedirs(out_dir, exist_ok = True)

cnt = 0
for i in range(imgs_ave.shape[0]):
    
    # threshed
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    cnt += 1

    


# shOPA1

## cropped_001

### pred_001

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_001/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_001/cristae_active_learning/cristae_unet_resblock_001_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_xy_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_yz_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_zx_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_xy_001/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_yz_001/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_zx_001/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_001/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_xy_001/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_yz_001/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_zx_001/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_001/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_001/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_001/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_001/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/manually_cristae_001/'
os.makedirs(out_dir, exist_ok = True)

### pred_002

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_001/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_001/cristae_active_learning/cristae_unet_resblock_002_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_xy_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_yz_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_zx_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_xy_002/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_yz_002/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_zx_002/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_002/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_xy_002/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_yz_002/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_zx_002/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_002/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_002/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_002/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_002/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/manually_cristae_002/'
os.makedirs(out_dir, exist_ok = True)

### pred_003

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_001/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_001/cristae_active_learning/cristae_unet_resblock_003_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_xy_003/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_yz_003/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_zx_003/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_xy_003/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_yz_003/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_zx_003/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_003/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_xy_003/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_yz_003/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_zx_003/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_003/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_003/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_003/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_003/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/manually_cristae_003/'
os.makedirs(out_dir, exist_ok = True)

### pred_004

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_001/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_001/cristae_active_learning/cristae_unet_resblock_004_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_xy_004/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_yz_004/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_zx_004/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_xy_004/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_yz_004/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_zx_004/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_004/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_xy_004/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_yz_004/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_zx_004/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_004/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_004/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_004/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_004/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/manually_cristae_004/'
os.makedirs(out_dir, exist_ok = True)

### pred_005

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_001/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_001/cristae_active_learning/cristae_unet_resblock_005_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_xy_005/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_yz_005/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_zx_005/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_xy_005/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_yz_005/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_zx_005/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_005/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_xy_005/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_yz_005/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_zx_005/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_005/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_005/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_005/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_005/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/manually_cristae_005/'
os.makedirs(out_dir, exist_ok = True)

### pred_006

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_001/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_001/cristae_active_learning/cristae_unet_resblock_006_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_xy_006/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_yz_006/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_zx_006/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_xy_006/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_yz_006/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_zx_006/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_006/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_xy_006/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_yz_006/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_zx_006/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_006/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_006/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_006/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_006/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/manually_cristae_006/'
os.makedirs(out_dir, exist_ok = True)

### pred_007

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_001/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_001/cristae_active_learning/cristae_unet_resblock_007_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_xy_007/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_yz_007/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_zx_007/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_xy_007/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_yz_007/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_zx_007/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_007/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_xy_006/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_yz_006/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_zx_006/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_006/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_007/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_007/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_007/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/manually_cristae_007/'
os.makedirs(out_dir, exist_ok = True)

### pred_final

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_001/resize_10x10x10')

ori_image_shape = ori_imgs.shape
del ori_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_xy_007/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_yz_007/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_zx_007/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx

imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_final/lamellar'
os.makedirs(out_dir, exist_ok = True)

cnt = 0
for i in range(imgs_ave.shape[0]):
    
    # threshed
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_xy_007/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_yz_007/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/cristae_pred_zx_007/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx

imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_final/tubular'
os.makedirs(out_dir, exist_ok = True)

cnt = 0
for i in range(imgs_ave.shape[0]):
    
    # threshed
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    cnt += 1

    


## cropped_002

### pred_001

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_002/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_002/cristae_active_learning/cristae_unet_resblock_001_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_xy_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_yz_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_zx_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_xy_001/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_yz_001/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_zx_001/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_001/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_xy_001/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_yz_001/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_zx_001/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_001/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_001/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_001/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_001/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/manually_cristae_001/'
os.makedirs(out_dir, exist_ok = True)

### pred_002

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_002/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_002/cristae_active_learning/cristae_unet_resblock_002_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_xy_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_yz_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_zx_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_xy_002/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_yz_002/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_zx_002/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_002/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_xy_002/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_yz_002/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_zx_002/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_002/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_002/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_002/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_002/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/manually_cristae_002/'
os.makedirs(out_dir, exist_ok = True)

### pred_003

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_002/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_002/cristae_active_learning/cristae_unet_resblock_003_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_xy_003/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_yz_003/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_zx_003/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_xy_003/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_yz_003/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_zx_003/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_003/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_xy_003/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_yz_003/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_zx_003/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_003/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_003/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_003/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_003/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/manually_cristae_003/'
os.makedirs(out_dir, exist_ok = True)

### pred_004

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_002/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_002/cristae_active_learning/cristae_unet_resblock_004_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_xy_004/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_yz_004/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_zx_004/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_xy_004/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_yz_004/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_zx_004/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_004/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_xy_004/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_yz_004/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_zx_004/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_004/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_004/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_004/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_004/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/manually_cristae_004/'
os.makedirs(out_dir, exist_ok = True)

### pred_005

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_002/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_002/cristae_active_learning/cristae_unet_resblock_005_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_xy_005/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_yz_005/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_zx_005/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_xy_005/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_yz_005/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_zx_005/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_005/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_xy_005/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_yz_005/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_zx_005/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_005/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_005/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_005/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_005/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/manually_cristae_005/'
os.makedirs(out_dir, exist_ok = True)

### pred_006

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_002/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_002/cristae_active_learning/cristae_unet_resblock_006_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_xy_006/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_yz_006/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_zx_006/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_xy_006/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_yz_006/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_zx_006/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_006/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_xy_006/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_yz_006/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_zx_006/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_006/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_006/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_006/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_006/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/manually_cristae_006/'
os.makedirs(out_dir, exist_ok = True)

### pred_007

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_002/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_002/cristae_active_learning/cristae_unet_resblock_007_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_xy_007/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_yz_007/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_zx_007/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_xy_007/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_yz_007/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_zx_007/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_007/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_xy_007/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_yz_007/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_zx_007/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_007/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_007/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_007/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_007/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/manually_cristae_007/'
os.makedirs(out_dir, exist_ok = True)

### pred_final

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_002/resize_10x10x10')

ori_image_shape = ori_imgs.shape
del ori_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_xy_007/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_yz_007/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_zx_007/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx

imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_final/lamellar'
os.makedirs(out_dir, exist_ok = True)

cnt = 0
for i in range(imgs_ave.shape[0]):
    
    # threshed
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_xy_007/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_yz_007/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/cristae_pred_zx_007/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx

imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_final/tubular'
os.makedirs(out_dir, exist_ok = True)

cnt = 0
for i in range(imgs_ave.shape[0]):
    
    # threshed
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    cnt += 1

    


## cropped_003

### pred_001

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_003/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_003/cristae_active_learning/cristae_unet_resblock_001_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/cristae_pred_xy_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/cristae_pred_yz_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/cristae_pred_zx_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/cristae_pred_xy_001/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/cristae_pred_yz_001/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/cristae_pred_zx_001/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/merged_cristae_001/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/cristae_pred_xy_001/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/cristae_pred_yz_001/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/cristae_pred_zx_001/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/merged_cristae_001/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/merged_cristae_001/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/merged_cristae_001/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/merged_cristae_001/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/manually_cristae_001/'
os.makedirs(out_dir, exist_ok = True)

### pred_002

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_003/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_003/cristae_active_learning/cristae_unet_resblock_002_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/cristae_pred_xy_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/cristae_pred_yz_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/cristae_pred_zx_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/cristae_pred_xy_002/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/cristae_pred_yz_002/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/cristae_pred_zx_002/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/merged_cristae_002/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/cristae_pred_xy_002/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/cristae_pred_yz_002/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/cristae_pred_zx_002/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/merged_cristae_002/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/merged_cristae_002/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/merged_cristae_002/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/merged_cristae_002/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/manually_cristae_002/'
os.makedirs(out_dir, exist_ok = True)

### pred_final

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_003/resize_10x10x10')

ori_image_shape = ori_imgs.shape
del ori_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/cristae_pred_xy_002/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/cristae_pred_yz_002/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/cristae_pred_zx_002/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx

imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/merged_cristae_final/lamellar'
os.makedirs(out_dir, exist_ok = True)

cnt = 0
for i in range(imgs_ave.shape[0]):
    
    # threshed
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/cristae_pred_xy_002/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/cristae_pred_yz_002/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/cristae_pred_zx_002/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx

imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/merged_cristae_final/tubular'
os.makedirs(out_dir, exist_ok = True)

cnt = 0
for i in range(imgs_ave.shape[0]):
    
    # threshed
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    cnt += 1

    


## cropped_004

### pred_001

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_004/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_004/cristae_active_learning/cristae_unet_resblock_001_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/cristae_pred_xy_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/cristae_pred_yz_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/cristae_pred_zx_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/cristae_pred_xy_001/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/cristae_pred_yz_001/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/cristae_pred_zx_001/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/merged_cristae_001/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/cristae_pred_xy_001/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/cristae_pred_yz_001/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/cristae_pred_zx_001/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/merged_cristae_001/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/merged_cristae_001/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/merged_cristae_001/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/merged_cristae_001/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/manually_cristae_001/'
os.makedirs(out_dir, exist_ok = True)

### pred_002

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_004/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_004/cristae_active_learning/cristae_unet_resblock_002_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/cristae_pred_xy_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/cristae_pred_yz_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/cristae_pred_zx_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/cristae_pred_xy_002/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/cristae_pred_yz_002/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/cristae_pred_zx_002/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/merged_cristae_002/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/cristae_pred_xy_002/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/cristae_pred_yz_002/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/cristae_pred_zx_002/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/merged_cristae_002/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/merged_cristae_002/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/merged_cristae_002/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/merged_cristae_002/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/manually_cristae_002/'
os.makedirs(out_dir, exist_ok = True)

### pred_final

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_004/resize_10x10x10')

ori_image_shape = ori_imgs.shape
del ori_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/cristae_pred_xy_002/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/cristae_pred_yz_002/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/cristae_pred_zx_002/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx

imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/merged_cristae_final/lamellar'
os.makedirs(out_dir, exist_ok = True)

cnt = 0
for i in range(imgs_ave.shape[0]):
    
    # threshed
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/cristae_pred_xy_002/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/cristae_pred_yz_002/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/cristae_pred_zx_002/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx

imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/merged_cristae_final/tubular'
os.makedirs(out_dir, exist_ok = True)

cnt = 0
for i in range(imgs_ave.shape[0]):
    
    # threshed
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    cnt += 1

    


## cropped_005

### pred_001

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_005/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_005/cristae_active_learning/cristae_unet_resblock_001_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_pred_xy_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_pred_yz_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_pred_zx_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_pred_xy_001/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_pred_yz_001/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_pred_zx_001/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/merged_cristae_001/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/cristae_pred_xy_001/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_pred_yz_001/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_pred_zx_001/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/merged_cristae_001/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/merged_cristae_001/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/merged_cristae_001/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/merged_cristae_001/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/manually_cristae_001/'
os.makedirs(out_dir, exist_ok = True)

### pred_002

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_005/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_005/cristae_active_learning/cristae_unet_resblock_002_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_pred_xy_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_pred_yz_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_pred_zx_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_pred_xy_002/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_pred_yz_002/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_pred_zx_002/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/merged_cristae_002/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/cristae_pred_xy_002/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_pred_yz_002/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_pred_zx_002/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/merged_cristae_002/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/merged_cristae_002/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/merged_cristae_002/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/merged_cristae_002/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/manually_cristae_002/'
os.makedirs(out_dir, exist_ok = True)

### pred_003

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_005/resize_10x10x10')
mito_imgs , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/devided_mito//')

model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_005/cristae_active_learning/cristae_unet_resblock_003_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=3
filenames = _

masked_imgs = make_white_mask_ori_img(ori_imgs, mito_imgs)

In [ ]:
seped_xy_imgs = divide_imgs(masked_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1) + np.sum(seped_xy_imgs[i] == 1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_pred_xy_003/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test, target_imgs, target_imgs_lst


seped_yz_imgs = divide_imgs(masked_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1) + np.sum(seped_yz_imgs[i] == 1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_pred_yz_003/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(masked_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1) + np.sum(seped_zx_imgs[i] == 1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_pred_zx_003/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs, mito_imgs, masked_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_pred_xy_003/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_pred_yz_003/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_pred_zx_003/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/merged_cristae_003/lamellar'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/cristae_pred_xy_003/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_pred_yz_003/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_pred_zx_003/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx


imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/merged_cristae_003/tubular'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
lamellar_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/merged_cristae_003/lamellar')
tubular_imgs, _ = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/merged_cristae_003/tubular')

merged_imgs = lamellar_imgs * 255 + tubular_imgs * 255 * 2

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/merged_cristae_003/merged_lamellar_and_tubular'
os.makedirs(out_dir, exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_dir}/{str(i).zfill(4)}.png",
        merged_imgs[i]
    )
    
    
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/manually_cristae_003/'
os.makedirs(out_dir, exist_ok = True)

### pred_final

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_005/resize_10x10x10')

ori_image_shape = ori_imgs.shape
del ori_imgs

ori_image_shape

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_pred_xy_003/1')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_pred_yz_003/1')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_pred_zx_003/1')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx

imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/merged_cristae_final/lamellar'
os.makedirs(out_dir, exist_ok = True)

cnt = 0
for i in range(imgs_ave.shape[0]):
    
    # threshed
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    cnt += 1

    


In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_pred_xy_003/2')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_pred_yz_003/2')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray_with_gaussian('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/cristae_pred_zx_003/2')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx

imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/merged_cristae_final/tubular'
os.makedirs(out_dir, exist_ok = True)

cnt = 0
for i in range(imgs_ave.shape[0]):
    
    # threshed
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    cnt += 1

    
